In [ ]:
pip install pandas folium

In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

file_path = '/Users/usernamemateo/Downloads/RIJA _ Iniciativas de Participacion Ciu dadana en el Poder Judicial - Mapeo.xlsx'

mapeo_casos_data = pd.read_excel(file_path, sheet_name='Mapeo de Casos')

# Manually specified coordinates for countries, including additional ones
country_coords = {
    # Existing and new countries combined
    'Argentina': (-38.4161, -63.6167),
    'Costa Rica': (9.7489, -83.7534),
    'Ecuador': (-1.8312, -78.1834),
    'Guatemala': (15.7835, -90.2308),
    'Paraguay': (-23.4425, -58.4438),
    'Peru': (-9.1900, -75.0152),
    'Colombia': (4.5709, -74.2973),
    'México': (23.6345, -102.5528),
    'Panamá': (8.5380, -80.7821),
    'Chile': (-35.6751, -71.5430),
    'Brasil': (-14.2350, -51.9253),
    'Honduras': (15.2000, -86.2419),
    'El Salvador': (13.7942, -88.8965),
    'Nicaragua': (12.8654, -85.2072),
    'República Dominicana': (18.7357, -70.1627),
    'Venezuela': (6.4238, -66.5897),
    'Uruguay': (-32.5228, -55.7658),
    'Georgia': (42.3154, 43.3569),
    'Estados Unidos': (37.0902, -95.7129),
    'Liberia': (6.4281, -9.4295),
    'Nigeria': (9.0820, 8.6753),
    'España': (40.4637, -3.7492),
    'Japón': (36.2048, 138.2529),
    'Tanzania': (-6.3690, 34.8888),
    'Albania': (41.1533, 20.1683),
    'Francia': (46.2276, 2.2137),
    'Bangladesh': (23.6850, 90.3563),
    'Jamaica': (18.1096, -77.2975),
    'Kosovo': (42.6026, 20.9029),
    'Ghana': (7.9465, -1.0232),
    'Martinique': (14.6415, -61.0242),
    'Eslovaquia': (48.6690, 19.6990),
    'Sudáfrica': (-30.5595, 22.9375),
    'Tailandia': (15.8700, 100.9925),
    'Puerto Rico': (18.2208, -66.5901),
    'Escocia': (56.4907, -4.2026),
    'Corea del Sur': (35.9078, 127.7669),
    'Australia': (-25.2744, 133.7751),
    'Kenya': (-0.0236, 37.9062),
    'Uganda': (1.3733, 32.2903),
    'Bolivia': (-16.2902, -63.5887),
    'Reino Unido': (55.3781, -3.4360)
}

# Apply coordinates
mapeo_casos_data['Latitude'] = mapeo_casos_data['País'].apply(lambda x: country_coords.get(x, (None, None))[0])
mapeo_casos_data['Longitude'] = mapeo_casos_data['País'].apply(lambda x: country_coords.get(x, (None, None))[1])

# Initialize Folium map
map = folium.Map(location=[20, 0], zoom_start=2, tiles='cartodbpositron')

# Define colors for the regions
region_colors = {
    'América Latina': 'blue',
    'Europa': 'green',
    'África': 'red',
    'Asia': 'purple',
    'Norteamérica': 'orange',
    'Oceanía': 'darkblue'
}

# Add MarkerCluster to manage the markers
marker_cluster = MarkerCluster().add_to(map)

# Add markers to the map
for idx, row in mapeo_casos_data.iterrows():
    if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
        folium.CircleMarker(
            location=(row['Latitude'], row['Longitude']),
            radius=5,
            popup=f"{row['País']}: {row['Iniciativa']}",
            color=region_colors.get(row['Región'], 'gray'),  # Default color if region is not in stated on the dictionary
            fill=True,
            fill_color=region_colors.get(row['Región'], 'gray'),
            fill_opacity=0.7
        ).add_to(marker_cluster)

# Save to HTML
map.save('interactive_map.html')


In [1]:
# Import necessary libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, FileLink

# Load data including coordinates (replace 'path_to_file' with actual path)
file_path = 'Mapeo_de_Casos_With_Coordinates.xlsx'
mapeo_de_casos = pd.read_excel(file_path)

# Function to generate map with chosen description type and filters
def generate_map(description_type, year_filter=None, country_filter=None, drivers_filter=None, save_path="map.html"):
    map_center = [4.0, -74.0]  # Centered around Latin America
    m = folium.Map(location=map_center, zoom_start=4, tiles='CartoDB positron')
    marker_cluster = MarkerCluster().add_to(m)

    # Apply filters
    filtered_data = mapeo_de_casos.copy()
    if year_filter:
        filtered_data = filtered_data[filtered_data['Año de Inicio'].isin(year_filter)]
    if country_filter:
        filtered_data = filtered_data[filtered_data['País'].isin(country_filter)]
    if drivers_filter:
        filtered_data = filtered_data[filtered_data['Impulsores'].isin(drivers_filter)]

    # Add markers based on filtered data
    for _, row in filtered_data.iterrows():
        if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
            coordinates = [row['Latitude'], row['Longitude']]
            
            # Select description type
            description = row['Description'] if description_type == "Full" else row['Description (Homogenized)']
            
            # Create pop-up content
            popup_content = f"""
            <b>Initiative:</b> {row['Nombre']}<br>
            <b>Country:</b> {row['País']}<br>
            <b>Year:</b> {row['Año de Inicio']}<br>
            <b>Drivers:</b> {row['Impulsores']}<br>
            <b>Description:</b> {description}<br>
            <b><a href='{row['Links / Recursos']}' target='_blank'>Technical Sheet</a></b>
            """
            
            folium.Marker(
                location=coordinates,
                popup=folium.Popup(popup_content, max_width=300),
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(marker_cluster)
    
    # Save map to specified path
    m.save(save_path)
    display(FileLink(save_path))

# Define different filter combinations
year_filters = [[2021], [2020, 2021]]  # Example years
country_filters = [["Argentina"], ["Argentina", "Brazil"]]  # Example countries
drivers_filters = [["Supreme Court"], ["Government", "Supreme Court"]]  # Example drivers

# Generating maps for each filter combination
description_types = ["Full", "Homogenized"]
filter_combinations = [
    # Single Filter
    {"year_filter": year_filters[0], "country_filter": None, "drivers_filter": None},
    {"year_filter": None, "country_filter": country_filters[0], "drivers_filter": None},
    {"year_filter": None, "country_filter": None, "drivers_filter": drivers_filters[0]},

    # Dual Filters
    {"year_filter": year_filters[0], "country_filter": country_filters[0], "drivers_filter": None},
    {"year_filter": year_filters[0], "country_filter": None, "drivers_filter": drivers_filters[0]},
    {"year_filter": None, "country_filter": country_filters[0], "drivers_filter": drivers_filters[0]},

    # All Filters Combined
    {"year_filter": year_filters[1], "country_filter": country_filters[1], "drivers_filter": drivers_filters[1]},
]

# Iterate over each combination and generate maps
for i, filters in enumerate(filter_combinations):
    for description_type in description_types:
        save_path = f"map_{description_type}_combo_{i+1}.html"
        generate_map(description_type=description_type,
                     year_filter=filters['year_filter'],
                     country_filter=filters['country_filter'],
                     drivers_filter=filters['drivers_filter'],
                     save_path=save_path)


/Users/usernamemateo/Downloads/RIJA/map_Full_combo_1.html

/Users/usernamemateo/Downloads/RIJA/map_Homogenized_combo_1.html

/Users/usernamemateo/Downloads/RIJA/map_Full_combo_2.html

/Users/usernamemateo/Downloads/RIJA/map_Homogenized_combo_2.html

/Users/usernamemateo/Downloads/RIJA/map_Full_combo_3.html

...

/Users/usernamemateo/Downloads/RIJA/map_Homogenized_combo_5.html

/Users/usernamemateo/Downloads/RIJA/map_Full_combo_6.html

/Users/usernamemateo/Downloads/RIJA/map_Homogenized_combo_6.html

/Users/usernamemateo/Downloads/RIJA/map_Full_combo_7.html

/Users/usernamemateo/Downloads/RIJA/map_Homogenized_combo_7.html

In [2]:
# Import necessary libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import ipywidgets as widgets

# Load data including coordinates (replace 'path_to_file' with actual path)
file_path = 'Mapeo_de_Casos_With_Coordinates.xlsx'
mapeo_de_casos = pd.read_excel(file_path)

# Function to generate map with chosen description type and filters
def create_interactive_map(description_type, year_filter=None, country_filter=None, drivers_filter=None):
    map_center = [4.0, -74.0]  # Centered around Latin America
    m = folium.Map(location=map_center, zoom_start=4, tiles='CartoDB positron')
    marker_cluster = MarkerCluster().add_to(m)

    # Apply filters based on user selections
    filtered_data = mapeo_de_casos.copy()
    if year_filter:
        filtered_data = filtered_data[filtered_data['Año de Inicio'].isin(year_filter)]
    if country_filter:
        filtered_data = filtered_data[filtered_data['País'].isin(country_filter)]
    if drivers_filter:
        filtered_data = filtered_data[filtered_data['Impulsores'].isin(drivers_filter)]

    # Add markers for each initiative with specified description type
    for _, row in filtered_data.iterrows():
        if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
            coordinates = [row['Latitude'], row['Longitude']]
            # Choose description type
            description = row['Description'] if description_type == "Full" else row['Description (Homogenized)']
            
            # Pop-up content
            popup_content = f"""
            <b>Initiative:</b> {row['Nombre']}<br>
            <b>Country:</b> {row['País']}<br>
            <b>Year:</b> {row['Año de Inicio']}<br>
            <b>Drivers:</b> {row['Impulsores']}<br>
            <b>Description:</b> {description}<br>
            <b><a href='{row['Links / Recursos']}' target='_blank'>Technical Sheet</a></b>
            """
            
            folium.Marker(
                location=coordinates,
                popup=folium.Popup(popup_content, max_width=300),
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(marker_cluster)
    
    return m

# Define interactive widgets for filter selection
years = sorted(mapeo_de_casos['Año de Inicio'].dropna().unique())
countries = sorted(mapeo_de_casos['País'].dropna().unique())
drivers = sorted(mapeo_de_casos['Impulsores'].dropna().unique())

year_filter_widget = widgets.SelectMultiple(options=years, description="Year")
country_filter_widget = widgets.SelectMultiple(options=countries, description="Country")
drivers_filter_widget = widgets.SelectMultiple(options=drivers, description="Drivers")

# Interactive function to update map based on widget selections
def update_map(description_type, year_filter, country_filter, drivers_filter):
    # Convert selections to lists
    year_filter = list(year_filter) if year_filter else None
    country_filter = list(country_filter) if country_filter else None
    drivers_filter = list(drivers_filter) if drivers_filter else None
    
    # Generate and display map
    map_display = create_interactive_map(description_type, year_filter, country_filter, drivers_filter)
    display(map_display)

# Tabs for Full and Homogenized map views
tab_namer = widgets.ToggleButtons(
    options=['Full Description', 'Homogenized Description'],
    description="Select Map Type:",
)

# Function to render map based on selected tab and filters
def render_map(tab_namer, year_filter, country_filter, drivers_filter):
    description_type = "Full" if tab_namer == 'Full Description' else "Homogenized"
    update_map(description_type, year_filter, country_filter, drivers_filter)

# Display the tabs, filters, and interactive map generation in the notebook
display(
    widgets.VBox([
        tab_namer,
        year_filter_widget,
        country_filter_widget,
        drivers_filter_widget,
        widgets.interactive(render_map, 
                            tab_namer=tab_namer, 
                            year_filter=year_filter_widget, 
                            country_filter=country_filter_widget, 
                            drivers_filter=drivers_filter_widget)
    ])
)


In [4]:
# Import necessary libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import ipywidgets as widgets

# Load data including coordinates (replace 'path_to_file' with actual path)
file_path = 'Mapeo_de_Casos_With_Coordinates.xlsx'
mapeo_de_casos = pd.read_excel(file_path)

# Function to create map with specified filters and description type
def create_map_with_filters(description_type, year_filter=None, country_filter=None, drivers_filter=None):
    map_center = [4.0, -74.0]  # Centered around Latin America
    m = folium.Map(location=map_center, zoom_start=4, tiles='CartoDB positron')
    marker_cluster = MarkerCluster().add_to(m)

    # Filter data
    filtered_data = mapeo_de_casos.copy()
    if year_filter:
        filtered_data = filtered_data[filtered_data['Año de Inicio'].isin(year_filter)]
    if country_filter:
        filtered_data = filtered_data[filtered_data['País'].isin(country_filter)]
    if drivers_filter:
        filtered_data = filtered_data[filtered_data['Impulsores'].isin(drivers_filter)]

    # Add markers with filtered data
    for _, row in filtered_data.iterrows():
        if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
            coordinates = [row['Latitude'], row['Longitude']]
            description = row['Description'] if description_type == "Full" else row['Description (Homogenized)']
            popup_content = f"""
            <b>Initiative:</b> {row['Nombre']}<br>
            <b>Country:</b> {row['País']}<br>
            <b>Year:</b> {row['Año de Inicio']}<br>
            <b>Drivers:</b> {row['Impulsores']}<br>
            <b>Description:</b> {description}<br>
            <b><a href='{row['Links / Recursos']}' target='_blank'>Technical Sheet</a></b>
            """
            folium.Marker(
                location=coordinates,
                popup=folium.Popup(popup_content, max_width=300),
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(marker_cluster)
    
    return m

# Widgets for filters
years = sorted(mapeo_de_casos['Año de Inicio'].dropna().unique())
countries = sorted(mapeo_de_casos['País'].dropna().unique())
drivers = sorted(mapeo_de_casos['Impulsores'].dropna().unique())

# Filter widgets for Full Description Map
year_filter_full = widgets.SelectMultiple(options=years, description="Year")
country_filter_full = widgets.SelectMultiple(options=countries, description="Country")
drivers_filter_full = widgets.SelectMultiple(options=drivers, description="Drivers")

# Filter widgets for Homogenized Description Map
year_filter_homogenized = widgets.SelectMultiple(options=years, description="Year")
country_filter_homogenized = widgets.SelectMultiple(options=countries, description="Country")
drivers_filter_homogenized = widgets.SelectMultiple(options=drivers, description="Drivers")

# Functions to update each map
def update_full_map(year_filter, country_filter, drivers_filter):
    # Convert selections to lists
    year_filter = list(year_filter) if year_filter else None
    country_filter = list(country_filter) if country_filter else None
    drivers_filter = list(drivers_filter) if drivers_filter else None
    
    # Generate map with full descriptions
    map_full = create_map_with_filters("Full", year_filter, country_filter, drivers_filter)
    display(map_full)

def update_homogenized_map(year_filter, country_filter, drivers_filter):
    # Convert selections to lists
    year_filter = list(year_filter) if year_filter else None
    country_filter = list(country_filter) if country_filter else None
    drivers_filter = list(drivers_filter) if drivers_filter else None
    
    # Generate map with homogenized descriptions
    map_homogenized = create_map_with_filters("Homogenized", year_filter, country_filter, drivers_filter)
    display(map_homogenized)

# Display both maps side-by-side
display(widgets.HTML("<h2>Full Description Map</h2>"))
display(
    widgets.VBox([
        year_filter_full, 
        country_filter_full, 
        drivers_filter_full, 
        widgets.interactive(update_full_map, 
                            year_filter=year_filter_full, 
                            country_filter=country_filter_full, 
                            drivers_filter=drivers_filter_full)
    ])
)

display(widgets.HTML("<h2>Homogenized Description Map</h2>"))
display(
    widgets.VBox([
        year_filter_homogenized, 
        country_filter_homogenized, 
        drivers_filter_homogenized, 
        widgets.interactive(update_homogenized_map, 
                            year_filter=year_filter_homogenized, 
                            country_filter=country_filter_homogenized, 
                            drivers_filter=drivers_filter_homogenized)
    ])
)


HTML(value='<h2>Full Description Map</h2>')

HTML(value='<h2>Homogenized Description Map</h2>')